In [165]:
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots

In [166]:
import numpy as np
import pandas as pd


In [167]:
import pulp

In [168]:
f_conv_mon = 0.0008

In [169]:
def bmatrix(a):
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [170]:
C = np.array([[105, 1267, 1134, 2758, 515, 1350, 1311, 1750, 1565, 1137, 3110, 1365]]) * f_conv_mon

C = np.concatenate(( np.concatenate((C, np.zeros((1, 12))), axis=1), 
                     np.concatenate((np.zeros((1, 12)), np.ones((1, 12))), axis=1),
                     np.concatenate((np.ones((1, 12)), np.zeros((1, 12))), axis=1)
                    ), axis=0)

In [171]:
A = np.array([  [     1,       1,       1,       1,       1,       1,       0,       0,       0,       0,       0,       0],
                [     0,       0,       0,       0,       0,       0,       1,       1,       1,       1,       1,       1],
                [   100,      50,      50,      50,     100,     100,      50,     100,     100,     100,     100,     100],
                [   100,      50,      50,      25,      50,     150,      50,     100,     100,      50,      50,      50],
                [     0,       0,       0,       0,       0,      12,       0,       0,      12,       0,       0,       0],
                [     1,       1,       2,       1,       1,       3,       1,       2,       2,       2,       3,       1],
                [     6,      29,      32,      14,      54,      60,       0,       0,      18,      13,      29,       8],
                [     2,       0,      17,      23,       0,       0,      22,       6,      50,     102,      77,      31],
                [     0,       0,       0,       0,       0,      12,      35,      24,      80,       0,     112,      58],
                [     9,       8,       7,      34,      26,      22,       0,       0,      52,       0,      43,       0],
                [   1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5],
                [     1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1],
                [   0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5],
                [   0.5,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0],
                [   390,     355,     370,     430,     440,       0,     275,       0,     100,     133,     213,     100],
                [   275,       0,     470,     305,       0,       0,    1430,     188,    1015,     955,    1343,    1215],
                [     0,       0,     150,       0,       0,     750,     443,     840,    1683,       0,    1748,    1095],
                [   300,     295,     305,     303,     210,     860,       0,       0,       0,       0,       0,       0],
                [345000,  233000,  216000,  242000,  360000,  650000,  244000,  242000,  310000,  263000,  250000,  285000]])

In [172]:
A[18,:] = A[18,:] * f_conv_mon

In [173]:
np.set_printoptions(precision = 3, suppress = True)
print(bmatrix(A[14:20, 0:12]))


\begin{bmatrix}
  390. & 355. & 370. & 430. & 440. & 0. & 275. & 0. & 100. & 133.\\
  213. & 100.\\
  275. & 0. & 470. & 305. & 0. & 0. & 1430. & 188. & 1015. & 955.\\
  1343. & 1215.\\
  0. & 0. & 150. & 0. & 0. & 750. & 443. & 840. & 1683. & 0.\\
  1748. & 1095.\\
  300. & 295. & 305. & 303. & 210. & 860. & 0. & 0. & 0. & 0.\\
  0. & 0.\\
  276. & 186.4 & 172.8 & 193.6 & 288. & 520. & 195.2 & 193.6 & 248. & 210.4\\
  200. & 228.\\
\end{bmatrix}


In [174]:
A = np.concatenate((A, np.zeros(A.shape)), axis=1)

In [175]:
I = np.identity(12)
b = 113 # min 5%
p = -(10**10)

A = np.concatenate((A, np.concatenate((-I, b*I), axis=1)), axis=0)
A = np.concatenate((A, np.concatenate((I, p*I), axis=1)), axis=0)

In [176]:
b = np.array([2395,2260,419190,397560,7000,13075,50593,554800,562400,562400,103056,103056,103056,954,20699716,20929709,21159705,21159705,1445855000])

In [177]:
b[18] = b[18] * f_conv_mon

In [178]:
b = np.concatenate((b, np.zeros((A.shape[0] - b.shape[0]))), axis=0)

In [179]:
sig = lambda sigma, NUM_ITER: 1/(1 + np.exp(sigma*(np.linspace(0, NUM_ITER, NUM_ITER)-(NUM_ITER/2))))

alpha_lin = np.linspace(0, 1, 1000)
alpha_sig_004 = sig(-0.004, 1000)
alpha_sig_06 = sig(-0.06, 1000)

In [180]:
l_alpha = {'alpha_lin':     alpha_lin, 
           #'alpha_sig_004': alpha_sig_004, 
           #'alpha_sig_06':  alpha_sig_06,
           }

In [181]:
def create_var():
    var = np.array([])

    lbx = [0] * 12
    #lbx[0] = 1500

    lby = [0] * 12

    for i in range(1, 13):
        var = np.append(var, np.array([pulp.LpVariable("x{}".format(i), lowBound = lbx[i-1], upBound=None, cat='Integer')]))

    for i in range(1, 13):
        var = np.append(var, np.array([pulp.LpVariable("y{}".format(i), lowBound = lby[i-1], upBound=None, cat='Binary')]))
    
    return var

In [182]:
z1p = 9088.072
z1n = 0

_z1 = z1p-z1n

z2p = 12
z2n = 0

_z2 = z2p-z2n

z3p = 4655
z3n = 0

_z3 = z3p-z3n

In [183]:
def objective(alpha):
    var = create_var()

    Fs = np.array([])

    for c in C:
        Fs = np.append(Fs, c.dot(var))

    fo = alpha*((Fs[0] - z1n)/_z1) + (1 - alpha) * ((Fs[1] -z2n)/(_z2*2) + (Fs[2] - z3n)/(_z3*2))

    return (var, Fs, fo)

In [184]:
def objective2(alpha):
    var = create_var()

    Fs = np.array([])

    for c in C:
        Fs = np.append(Fs, c.dot(var))

    Fs = Fs.reshape((3, 1))

    fo = alpha.dot(Fs)

    return (var, Fs, fo[0][0])

In [185]:
def print_solution(sol, linearProblem, X):
    saida = "\nStatus = {} \nValue = {}".format(pulp.LpStatus[sol], pulp.value(linearProblem.objective))
    
    for i in range(0, X.shape[0]):
        saida += "\n{} = {}".format(X[i], pulp.value(X[i]))
    
    print(saida)

In [186]:
columns=["alpha_type", "iter", "alpha", "status"]
for i in range(1, 13):
    columns.append("X{}".format(i))
for i in range(1, 13):
    columns.append("Y{}".format(i))
columns = columns + ["obj_value", "f1", "f2", "f3"]

In [187]:
solution_lst = []
for key in l_alpha:
    aph = l_alpha[key]
    for i_alpha in range(0, len(aph)):            
        alpha = aph[i_alpha]
        d_var, F, f = objective(alpha)
        model = pulp.LpProblem("First_Model", pulp.LpMaximize)
        #model = pulp.LpProblem("First_Model", pulp.LpMinimize)

        model += f 

        Ax = A.dot(d_var)

        for i in range(0, Ax.shape[0]):
            model += Ax[i] <= b[i]

        solution = model.solve(pulp.GLPK())
        #solution = model.solve()

        X = np.array([pulp.value(i) for i in d_var])

        fs = C.dot(X)

        solution_lst.append([key,
                            i_alpha,
                            alpha,
                            str(pulp.LpStatus[solution])] +
                            X.tolist() +
                            [pulp.value(model.objective),
                            fs[0],
                            fs[1],
                            fs[2],
                            ])
    #print_solution(sol=solution, linearProblem=model, X=d_var)

In [188]:
dt_all = pd.DataFrame(solution_lst, columns=columns)

In [189]:
dt_all

,alpha_type,iter,alpha,status,X1,X2,X3,X4,X5,X6,...,Y7,Y8,Y9,Y10,Y11,Y12,obj_value,f1,f2,f3
0,alpha_lin,0,0.000000,Optimal,1775,168,113,113,113,113,...,1,1,1,1,1,1,1.000000,3474.7328,12.0,4655.0
1,alpha_lin,1,0.001001,Optimal,509,113,113,1434,113,113,...,1,1,1,1,1,1,0.999741,6732.6952,12.0,4655.0
2,alpha_lin,2,0.002002,Optimal,509,113,113,1434,113,113,...,1,1,1,1,1,1,0.999481,6732.6952,12.0,4655.0
3,alpha_lin,3,0.003003,Optimal,509,113,113,1434,113,113,...,1,1,1,1,1,1,0.999222,6732.6952,12.0,4655.0
4,alpha_lin,4,0.004004,Optimal,509,113,113,1434,113,113,...,1,1,1,1,1,1,0.998962,6732.6952,12.0,4655.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,alpha_lin,995,0.995996,Optimal,0,0,0,2395,0,0,...,0,1,0,0,1,0,0.998498,9088.0720,3.0,4655.0
996,alpha_lin,996,0.996997,Optimal,0,0,0,2395,0,0,...,0,1,0,0,1,0,0.998874,9088.0720,3.0,4655.0
997,alpha_lin,997,0.997998,Optimal,0,0,0,2395,0,0,...,0,1,0,0,1,0,0.999249,9088.0720,3.0,4655.0
998,alpha_lin,998,0.998999,Optimal,0,0,0,2395,0,0,...,0,1,0,0,1,0,0.999625,9088.0720,3.0,4655.0


In [190]:
dt = dt_all[dt_all.alpha_type == 'alpha_lin']

In [191]:
#sns.scatterplot(data=dt, x = 'alpha', y='obj_value')

fig = px.scatter(dt, 
              x ='alpha', 
              y='obj_value', 
              title='Objective Function (z<sub>1y</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      'obj_value': 'z'})
                      
fig.update_traces(marker=dict(color='rgb(115, 115, 115)',
                              size=3
                              )
                    )

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
#fig['data'][0]['line']['color']='rgb(115, 115, 115)'
#fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl1_z_vs_lambda.png')

In [192]:
#sns.lineplot(data=dt, x = 'alpha', y='f1')

#sns.lineplot(data=dt, x = 'alpha', y='f2')

#sns.lineplot(data=dt, x = 'alpha', y='f3')

fig = px.line(dt, 
              x ='alpha', 
              y='f1', 
              title='Net Income of Farm (z<sub>1</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      #'alpha_type': 'λ',
                      'f1': 'z<sub>1</sub> (USD)'})
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        dtick=250,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
fig['data'][0]['line']['color']='rgb(115, 115, 115)'
fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl1_z1y_vs_lambda.png')

In [193]:
#sns.lineplot(data=dt, x = 'alpha', y='f2')

#sns.lineplot(data=dt, x = 'alpha', y='f3')

fig = px.line(dt, 
              x ='alpha', 
              y='f2', 
              title='Number of Different Crops Grown (z<sub>2</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      'f2': 'z<sub>2</sub>'})
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
fig['data'][0]['line']['color']='rgb(115, 115, 115)'
fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl1_z2_vs_lambda.png')

In [194]:
#sns.lineplot(data=dt, x = 'alpha', y='f3')

fig = px.line(dt, 
              x ='alpha', 
              y='f3', 
              title='Used Area (z<sub>3</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      'f3': 'z<sub>3</sub>'})
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
fig['data'][0]['line']['color']='rgb(115, 115, 115)'
fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl1_z3_vs_lambda.png')

In [195]:
dt_norn = dt[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10', 'Y11', 'Y12']]

dt_norn=((dt_norn-dt_norn.min())/(dt_norn.max()-dt_norn.min()))

#dt_norn=((dt_norn)/(dt_norn.max()-dt_norn.min()))

dt_norn['iter'] = dt['iter']
dt_norn['alpha'] = dt['alpha']

dt_norn['f1'] = round(dt['f1']/100000) #'f1', 'f2', 'f3'
dt_norn['f2'] = dt['f2']
dt_norn['f3'] = dt['f3']
dt_norn['obj_value'] =  dt['obj_value'] 


In [196]:
#cm = color_continuous_scale=px.colors.diverging.Tealrose,
cm = ['rgb(77,77,77)', 'rgb(153, 153, 153)', 'rgb(186,186,186)', 'rgb(244,165,130)', 'rgb(214,96,77)','rgb(230, 39, 23)']
cm = ['rgb(77,77,77)', 'rgb(153, 153, 153)', 'rgb(214,96,77)','rgb(230, 39, 23)']

https://plotly.com/python/parallel-coordinates-plot/
https://plotly.com/python-api-reference/generated/plotly.express.parallel_coordinates.html

In [197]:
from turtle import width


fig1 = px.parallel_coordinates(dt, 
                             dimensions=['alpha','X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12'],
                             color="f1", 
                             color_continuous_scale=cm,
                             labels={   'alpha':'λ', 
                                        'f1': 'z<sub>1</sub> (USD)',
                                        'X1':'x<sub>1 (Wheat)', 
                                        'X2':'x<sub>2 (Beetroot)', 
                                        'X3':'x<sub>3 (Cloves)', 
                                        'X4':'x<sub>4 (Broad Bean)', 
                                        'X5':'x<sub>5 (Green Onion)', 
                                        'X6':'x<sub>6 (Autumn Potatoes)', 
                                        'X7':'x<sub>7 (Sunflower)', 
                                        'X8':'x<sub>8 (Maize)', 
                                        'X9':'x<sub>9 (Tomatoes)', 
                                        'X10':'x<sub>10 (Cucumbers)', 
                                        'X11':'x<sub>11 (Okra)', 
                                        'X12':'x<sub>12 (Watermelon)'},
                             color_continuous_midpoint=2,
                             range_color = [dt.f1.min(), dt.f1.max()])

fig1.show()

#fig1.write_image('images/fig_mdl1_parallel_coordinates_z1.png', width=1400, height=600)

In [198]:
fig2 = px.parallel_coordinates(dt, 
                             dimensions=['alpha','X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12'],
                             color="f2", 
                             color_continuous_scale=cm,
                             labels={   'alpha':'λ', 
                                        'f2': 'z<sub>2</sub>',
                                        'X1':'x<sub>1 (Wheat)', 
                                        'X2':'x<sub>2 (Beetroot)', 
                                        'X3':'x<sub>3 (Cloves)', 
                                        'X4':'x<sub>4 (Broad Bean)', 
                                        'X5':'x<sub>5 (Green Onion)', 
                                        'X6':'x<sub>6 (Autumn Potatoes)', 
                                        'X7':'x<sub>7 (Sunflower)', 
                                        'X8':'x<sub>8 (Maize)', 
                                        'X9':'x<sub>9 (Tomatoes)', 
                                        'X10':'x<sub>10 (Cucumbers)', 
                                        'X11':'x<sub>11 (Okra)', 
                                        'X12':'x<sub>12 (Watermelon)'},
                             color_continuous_midpoint=2,
                             range_color = [dt.f2.min(), dt.f2.max()])
fig2.show()

#fig2.write_image('images/fig_mdl1_parallel_coordinates_z2.png', width=1400, height=600)